# ETL & Analysis

In [33]:
import os
import json
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd

from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np

In [15]:
# load environment
postgres_config = None
with open(os.path.join(os.path.expanduser('~'), '.postgres_connection.json'), 'r') as postgres_config:
    postgres_config = json.load(postgres_config)


In [16]:
# create db connection
dbname = postgres_config['database']
user = postgres_config['user']
password = postgres_config['password']
host = postgres_config['host']

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}/{dbname}?client_encoding=utf8")

### helper functions

In [34]:
def get_file_contents(file_name):
    """
    Returns Pandas Dataframe
    """
    df = None
    with open(os.path.join('.', 'data', file_name), 'r', encoding='utf8') as fd:
        df = pd.read_csv(fd)
    return df

def scrape():
    """
    This scrapes the data from a brewery listing 
    """
    # browser = init_browser()
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)

    #listings = {}

    url = "https://beerme.com/brewerylist.php"
    browser.visit(url)

    html = browser.html
    soup = BeautifulSoup(html, "html.parser")

    # Quit the browser
    browser.quit()

    return soup.find(id="content").find('ul').find_all('ul')

###  Brewery Data Clean-Up

In [36]:
brewery_data = []

# load data from scrape function
for entry in scrape():
    brewery_raw_entry = entry.text.split('\n')
    brewery_data.append(brewery_raw_entry)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Get LATEST driver version for 90.0.4430
Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\evera\.wdm\drivers\chromedriver\win32\90.0.4430.24]


In [44]:
# clean up
flat_brewery = [item for sublist in brewery_data for item in sublist]
brewdata2 = []

for x in flat_brewery:
    entry = x.split(',')
    breweryname = entry[0]
    country = entry[-1]
    brewdata2.append({'brewery':breweryname, 'country':country})

brewdata_df = pd.DataFrame(brewdata2)
brewdata_df['brewery'].replace('', np.nan, inplace=True)
brewdata_df.dropna(subset=['brewery'], inplace=True)
brewdata_df.head()

,brewery,country
2,Tschöams Biere,Austria
5,'A Magara,Italy
6,'Nora,Italy
7,'t Hofbrouwerijke,Belgium
10,(512) Brewing Company,United States


In [ ]:
# export to csv and sql
brewdata_df.to_csv(os.path.join('.', 'data', 'brewery_country.csv'))
beer_reviews_df.to_sql('brewery_country', engine, if_exists='replace', index=False, method = 'multi')

### Beer Reviews

In [18]:
# load beer reviews
beer_reviews_df = get_file_contents('beer_reviews.csv')

In [19]:
# clean the data
beer_reviews_df.beer_abv = beer_reviews_df.beer_abv.replace("", None)
beer_reviews_df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [ ]:
# load data to table
beer_reviews_df.to_sql('beerreviews', engine, if_exists='replace', index=False, method = 'multi')

## happiness and alcohol consumption

In [20]:
# load alcohol consumption
alcohol_consumption_df = get_file_contents('HappinessAlcoholConsumption.csv')
alcohol_consumption_df.head()

,Country,Region,Hemisphere,HappinessScore,HDI,GDP_PerCapita,Beer_PerCapita,Spirit_PerCapita,Wine_PerCapita
0,Denmark,Western Europe,north,7.526,928,53.579,224,81,278
1,Switzerland,Western Europe,north,7.509,943,79.866,185,100,280
2,Iceland,Western Europe,north,7.501,933,60.530,233,61,78
3,Norway,Western Europe,north,7.498,951,70.890,169,71,129
4,Finland,Western Europe,north,7.413,918,43.433,263,133,97


In [21]:
#clean up columns
alcohol_consumption_df = alcohol_consumption_df.rename(columns={
    'Country': 'country',
    'Region': 'region',
    'Hemisphere': 'hemisphere',
    'HappinessScore': 'happiness_score',
    'HDI': 'hdi',
    'GDP_PerCapita': 'gdp_per_capita',
    'Beer_PerCapita': 'beer_per_capita',
    'Spirit_PerCapita': 'spirit_per_capita',
    'Wine_PerCapita': 'wine_per_capita'
}).copy()
alcohol_consumption_df.head()

,country,region,hemisphere,happiness_score,hdi,gdp_per_capita,beer_per_capita,spirit_per_capita,wine_per_capita
0,Denmark,Western Europe,north,7.526,928,53.579,224,81,278
1,Switzerland,Western Europe,north,7.509,943,79.866,185,100,280
2,Iceland,Western Europe,north,7.501,933,60.530,233,61,78
3,Norway,Western Europe,north,7.498,951,70.890,169,71,129
4,Finland,Western Europe,north,7.413,918,43.433,263,133,97


In [41]:
# load data to sql
alcohol_consumption_df.to_sql('alcohol_consumption', engine, if_exists='replace', index=False, method = 'multi')

## breweries in countries

In [25]:
# load alcohol consumption
brewery_country_df = get_file_contents('brewery_country.csv')
brewery_country_df = brewery_country_df[['brewery', 'country']] \
    .rename(columns={
        'brewery': 'brewery_name'
    })
brewery_country_df.head()

,brewery_name,country
0,Tschöams Biere,Austria
1,'A Magara,Italy
2,'Nora,Italy
3,'t Hofbrouwerijke,Belgium
4,(512) Brewing Company,United States


In [28]:
# merge dataframes for anaylysis
brewery_complete_set = pd.merge(beer_reviews_df.set_index('brewery_name'),
         brewery_country_df.set_index('brewery_name'),
         on='brewery_name'
    )

In [29]:
brewery_complete_set.head()

,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,country
brewery_name,,,,,,,,,,,,,
Hoppin' Frog Brewery,14879,1233471180,4.0,4.0,4.0,Ummagum,American IPA,4.0,4.0,Hoppin' To Heaven IPA,6.8,33624,United States
Hoppin' Frog Brewery,14879,1320811507,4.0,4.0,4.0,baconbourbon,Russian Imperial Stout,4.5,4.0,Barrel Aged B.O.R.I.S. Oatmeal Imperial Stout,9.4,47695,United States
Hoppin' Frog Brewery,14879,1190158924,4.0,3.5,3.5,Wasatch,Belgian Strong Pale Ale,4.0,4.0,Gulden Fraug Belgian Style Ale,10.5,33927,United States
Hoppin' Frog Brewery,14879,1189922099,4.0,3.5,3.5,northyorksammy,Belgian Strong Pale Ale,4.0,3.5,Gulden Fraug Belgian Style Ale,10.5,33927,United States
Hoppin' Frog Brewery,14879,1177278180,3.0,3.0,3.0,AltBock,Belgian Strong Pale Ale,4.0,3.5,Gulden Fraug Belgian Style Ale,10.5,33927,United States


In [32]:
brewery_complete_set.groupby('country').mean()

,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
country,,,,,,,,,
Algeria,5005.666667,1.081737e+09,2.666667,2.500000,2.666667,2.500000,2.833333,4.650000,17077.666667
Argentina,15581.777070,1.243616e+09,3.347134,3.410828,3.471338,3.324841,3.318471,6.658333,37961.872611
Australia,13225.527163,1.250236e+09,3.518779,3.431254,3.553320,3.406774,3.395372,5.507148,41613.714286
Austria,1551.398625,1.200795e+09,3.500859,3.251718,3.474227,3.335911,3.371993,4.936298,14236.484536
Bangladesh,16023.000000,1.177787e+09,2.127907,2.069767,2.709302,2.139535,1.976744,5.267442,6308.348837
...,...,...,...,...,...,...,...,...,...
Tonga,13239.000000,1.229987e+09,3.500000,2.750000,3.000000,3.000000,3.000000,5.000000,34681.000000
Ukraine,17567.523077,1.248979e+09,3.500000,3.446154,3.753846,3.369231,3.315385,4.796721,46982.307692
United Kingdom,2230.923947,1.207126e+09,3.995880,3.841715,4.002354,3.904198,3.975314,5.686341,11180.411915
